In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self, taps, whatever, whatever1):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2*taps,whatever)   # 2 意味着每个complex数据，一个实部，一个虚部
        self.fc2 = nn.Linear(whatever,whatever1)
        self.fc3 = nn.Linear(whatever1,2)       # 2 意味着输出为complex数据
    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [3]:
in_data = np.array([1+0j,0-1j,0+1j,-1+0j])
in_real = in_data.real
in_imag = in_data.imag
in_complex = np.stack((in_real, in_imag))
print(in_complex)

[[ 1.  0.  0. -1.]
 [ 0. -1.  1.  0.]]


In [60]:
target = np.array([1+0j,0-1j,0+1j,-1+0j])
target_real = target.real
target_imag = target.imag
target_complex = np.stack((target_real, target_imag))
print(target_complex)

[[ 1.  0.  0. -1.]
 [ 0. -1.  1.  0.]]


In [61]:
net = Net(taps=2, whatever=10, whatever1=5)

In [62]:
# input: [[ 1.  0.  0. -1.]
#         [ 0. -1.  1.  0.]]
# taps = 2
# random_idx = 3
# Data_loader return [[0,-1]
#                     [1,0]]
#
# 如果load_input=False, 说明该Data_loader为target取数据 
# Data_loader return [[-2],[1]] 取最后一列
# 每次取一个随机index， 然后根据filter的taps， 取相关的前几个数据
# taps是几，就取几个数据，比如taps是2， 每次取到index， 就再取他前面的那个值， 一共取到两个complex数据

def Data_loader(dataset, taps, random_idx, load_input):
    if(load_input):
        length = dataset.shape[1]
        #random_idx = np.random.randint(length)    
        if(random_idx==0):
            # 如果index是0， 就取本身 和 末尾， 因为是周期性的数
            return np.concatenate((dataset[:,length-1:length],
                                   dataset[:,random_idx:random_idx+1]), axis=1)

        return dataset[:,random_idx-taps+1:random_idx+1]
        # return shape = (dataset.shape[0], taps)
        # dataset.shape[0] 就是complex number 的real 和 imag
    else:
        return dataset[:,random_idx]

In [63]:
taps = 2
random_idx = np.random.randint(in_complex.shape[1])
def flatten_complex(complex_data, taps):
    complex_data = complex_data.T.reshape(-1,2*taps)
    complex_data = complex_data.squeeze(0)
    return complex_data


feed_data = Data_loader(in_complex,taps, random_idx, True)
print(feed_data)
feed_data_0 = feed_data.reshape(-1, 2*taps)
feed_data_0 = feed_data_0.squeeze(0)
feed_data1 = flatten_complex(feed_data, taps)
print(feed_data_0, feed_data1)


[[ 0. -1.]
 [ 1.  0.]]
[ 0. -1.  1.  0.] [ 0.  1. -1.  0.]


In [64]:
in_data = torch.Tensor(feed_data1)
out = net(in_data)

In [65]:
random_idx = np.random.randint(in_complex.shape[1])
feed_data = Data_loader(in_complex, taps, random_idx, True)
target = Data_loader(target_complex, taps, random_idx, False)
print(feed_data)
feed_data = flatten_complex(feed_data, taps)
target = flatten_complex(target, 1)
print(feed_data, target)

[[-1.  1.]
 [ 0.  0.]]
[-1.  0.  1.  0.] [1. 0.]


In [66]:
import copy
def train_model(net, optimizer, num_epoch, save_path):
    i = 0
    best_loss = -1000
    best_model_wts = copy.deepcopy(net.state_dict())
    while i<num_epoch:
        optimizer.zero_grad()
        random_idx = np.random.randint(in_complex.shape[1])
        feed_data = Data_loader(in_complex, taps, random_idx, True)
        target = Data_loader(target_complex, taps, random_idx, False)
        feed_data = flatten_complex(feed_data, taps)
        target = flatten_complex(target, 1)
        feed_data = torch.Tensor(feed_data)
        target = torch.Tensor(target)
        out = net(feed_data)
        #print(out.size(), target.size())
        loss = torch.sum((target-out)**2)
        if(loss<best_loss):
            best_loss = loss
            best_model_wts = copy.deepcopy(net.state_dict())
            torch.save(model.state_dict(),save_path)
        print("loss is\n\n{}\n\n"
              "input is \n\n{}\n\n"
              "predict is \n\n{}\n\n"
              "target is \n\n{}\n\n".format(loss, feed_data, out, target))
        loss.backward()
        optimizer.step()
        i = i+1
    #net.load_state_dict(best_model_wts)
    return net

In [67]:
import torch.optim as optim
net = Net(taps=2, whatever=10, whatever1=5)
optimizer = optim.SGD(net.parameters(), lr=0.009, momentum= 0.9)

In [72]:
trained_model = train_model(net, optimizer, 20, 'FIR_dict')

loss is

1.4210854715202004e-14

input is 

tensor([-1.,  0.,  1.,  0.])

predict is 

tensor([ 1.0000, -0.0000], grad_fn=<ThAddBackward>)

target is 

tensor([1., 0.])


loss is

0.0

input is 

tensor([ 0., -1.,  0.,  1.])

predict is 

tensor([0., 1.], grad_fn=<ThAddBackward>)

target is 

tensor([0., 1.])


loss is

1.4210854715202004e-14

input is 

tensor([-1.,  0.,  1.,  0.])

predict is 

tensor([ 1.0000, -0.0000], grad_fn=<ThAddBackward>)

target is 

tensor([1., 0.])


loss is

1.4210854715202004e-14

input is 

tensor([-1.,  0.,  1.,  0.])

predict is 

tensor([ 1.0000, -0.0000], grad_fn=<ThAddBackward>)

target is 

tensor([1., 0.])


loss is

0.0

input is 

tensor([ 0., -1.,  0.,  1.])

predict is 

tensor([0., 1.], grad_fn=<ThAddBackward>)

target is 

tensor([0., 1.])


loss is

0.0

input is 

tensor([ 1.,  0.,  0., -1.])

predict is 

tensor([ 0., -1.], grad_fn=<ThAddBackward>)

target is 

tensor([ 0., -1.])


loss is

0.0

input is 

tensor([ 0., -1.,  0.,  1.])

pr

In [73]:
data = torch.Tensor(np.array([1,1,1,1]))

In [74]:
out = trained_model(data)

In [75]:
out

tensor([ 0.8499, -0.4050], grad_fn=<ThAddBackward>)